# Image Search Engine
- Provides search functionalty using images and text
- Training Data is available at [Vision.cs](http://vision.cs.uiuc.edu/pascal-sentences/)

In [ ]:
from PIL import Image
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras_preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from utils import *
from image_search_engine import image_search_engine
import numpy as np
import inspect
import os
import numpy as np
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

In [ ]:
# Intialize matplotlib parameters

params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'}

plt.rcParams.update(params)
%matplotlib inline

# pandas display data frames as tables
from IPython.display import display, HTML

import warnings
warnings.filterwarnings('ignore')

In [ ]:

glove_model_path = "/Volumes/My Passport for Mac/model/glove.6B"
data_path = "/Volumes/My Passport for Mac/data/imagesearch/"
features_path = "/Volumes/My Passport for Mac/model/imagesearch/features"
file_mapping_path = "/Volumes/My Passport for Mac/model/imagesearch/filemapping"
custom_features_path = "/Volumes/My Passport for Mac/model/imagesearch/customfeatures"
custom_features_file_mapping_path = "/Volumes/My Passport for Mac/model/imagesearch/customfilemapping"

In [ ]:
model = image_search_engine.load_headless_pretrained_model()

In [ ]:

print(model.summary())

In [ ]:
images, vectors, image_paths, word_vectors = load_images_vectors_paths(glove_model_path, data_path)

In [ ]:
generate_image_features = True
model = image_search_engine.load_headless_pretrained_model()
if generate_image_features:
    images_features, file_index = image_search_engine.generate_features(image_paths, model)
    image_search_engine.save_features(features_path, images_features, file_mapping_path, file_index)
else:
    print("loading image features from disk")
    images_features, file_index = image_search_engine.load_features(features_path, file_mapping_path)

In [ ]:

print(file_index[100])
print(images_features[100])

In [ ]:

image_index = image_search_engine.index_features(images_features)

In [ ]:
#print(file_index[200])
#results = image_search_engine.search_index_by_key(200, image_index, file_index)

In [ ]:
#print(results)
#displayImages(results)
# Index 284 is the index for the Siamese cat class in Imagenet
#weighted_features = image_search_engine.get_weighted_features(200, images_features)
#weighted_index = image_search_engine.index_features(weighted_features)
#weighted_results = image_search_engine.search_index_by_key(200, weighted_index, file_index)
#displayImages(weighted_results)

### Image Search by Feature Vector

In [ ]:
#image_path = "/Volumes/My Passport for Mac/data/imagesearch/aeroplane/2008_000716.jpg"
#image_path = "/Volumes/My Passport for Mac/data/imagesearch/bird/2008_000095.jpg"
image_path = "/Volumes/My Passport for Mac/data/imagesearch/boat/2008_004014.jpg"
displayImagebyPath(image_path)

In [ ]:
fv = image_search_engine.get_feature_vector(model, image_path)
results = image_search_engine.search_index_by_value(fv, image_index, file_index)
displayImages(results)

In [ ]:
print(word_vectors["bottle"])

In [ ]:
word_index, word_mapping = image_search_engine.build_word_index(word_vectors)

In [ ]:
fv = word_vectors["ocean"]
results = image_search_engine.search_index_by_value(fv, word_index, word_mapping)
print(results)

## Custom Build for Text-to-Image and Image-to-Text Search

In [ ]:
custom_model = image_search_engine.setup_custom_model()

In [ ]:
model_save_path = "/Volumes/My Passport for Mac/model/imagesearch/model2.hdf5"
model_load_oath = "/Volumes/My Passport for Mac/model/imagesearch/model2.hdf5"

train_model = True
if train_model:
    num_epochs = 50
    batch_size = 32
    print("Training for %s epochs, this might take a while, "
            "change train_model to False to load pre-trained model" % num_epochs)
    x, y = shuffle(images, vectors, random_state=2)
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)
    checkpointer = ModelCheckpoint(filepath='checkpoint.hdf5', verbose=1, save_best_only=True)
    history = custom_model.fit(X_train, y_train, validation_data=(X_test, y_test),
                         epochs=num_epochs, batch_size=batch_size, callbacks=[checkpointer])
    custom_model.save(model_save_path)
else:
    st.write("Loading model from `%s`" % model_load_path)
    custom_model = load_model(model_load_path)

## Visualization of Model Performance

In [ ]:
print(history.history["val_loss"])

In [ ]:
f, ax2 = plt.subplots(1, 1, figsize=(15, 5))
t = f.suptitle('Deep Neural Net Performance', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

epochs = list(range(1,51))
#ax1.plot(epochs, history.history['acc'], label='Train Accuracy')
#ax1.plot(epochs, history.history['val_accuracy'], label='Validation Accuracy')
#ax1.set_xticks(epochs)
#ax1.set_ylabel('Accuracy Value')
#ax1.set_xlabel('Epoch')
#ax1.set_title('Accuracy')
#l1 = ax1.legend(loc="best")

ax2.plot(epochs, history.history['loss'], label='Train Loss')
ax2.plot(epochs, history.history['val_loss'], label='Validation Loss')
ax2.set_xticks(epochs)
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")

In [ ]:
generate_custom_features = True
if generate_custom_features:
    hybrid_images_features, file_mapping = image_search_engine.generate_features(image_paths, custom_model)
    image_search_engine.save_features(custom_features_path, hybrid_images_features, custom_features_file_mapping_path,
                                file_mapping)
else:
    hybrid_images_features, file_mapping = image_search_engine.load_features(custom_features_path,
                                                                       custom_features_file_mapping_path)
image_index = image_search_engine.index_features(hybrid_images_features, dims=300)

### Image to Text Search

In [ ]:
print(file_mapping[200])
displayImagebyPath(file_mapping[200])

### Text to Image Search

In [ ]:
results = image_search_engine.search_index_by_value(word_vectors["cat"], image_index, file_mapping)
displayImages(results)

#### Search for Untrained words

In [ ]:
results = image_search_engine.search_index_by_value(word_vectors["ocean"], image_index, file_mapping)
displayImages(results)